In [1]:
import os
import csv

from PIL import Image as im

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split

import torchvision
from torchvision import transforms

import pytorch_lightning as pl
from tqdm import tqdm

In [2]:
class PLDataModule(pl.LightningDataModule):
    def __init__(self, data_root, img_size, batch_size=32, train_split=0.8):
        self.root = data_root
        self.metadata = self.get_metadata()
        self.img_size = img_size
        self.batch_size = batch_size
        self.train_split = train_split
        self.train_mean = [0.0, 0., 0.0]
        self.train_std = [1.0, 1.0, 1.0]
        self.common_transforms = transforms.Compose([
            transforms.Resize((img_size + 20, img_size + 20), 
                              transforms.InterpolationMode.BICUBIC),
            transforms.CenterCrop(img_size),
            transforms.ToTensor(),
            
        ])
        
    
    def prepare_data_per_node(self):
        # There is nothing to prepare.
        # The data should have already been downloaded and
        # extracted in the data_root that is passed to initiaize this class
        print(os.path.abspath(self.root))
        
    
    def setup(self, stage='fit'):
        data_dict = self.get_img_path_labels(stage)
        imgs_ct = len(data_dict['image_path'])
        imgs_tensor = torch.empty([imgs_ct, 3, self.img_size, self.img_size])
        labels_tensor = torch.empty(imgs_ct, dtype=int)
       
        for i in range(imgs_ct):
            img = im.open(data_dict['image_path'][i]).convert("RGB")
            img = self.common_transforms(img)
            imgs_tensor[i] = img
            labels_tensor[i] = data_dict['label'][i]
        
        if stage == 'fit':
            self.mean, self.std = self.compute_mean_std(imgs_tensor)
            imgs_tensor = transforms.functional.normalize(imgs_tensor, self.mean, self.std)
            final_dataset = tuple(map(lambda x, y: (x, y,), imgs_tensor, labels_tensor))
            train_split = torch.ceil(torch.tensor(imgs_ct * self.train_split)).to(int).item()
            self.train_set, self.val_set = random_split(final_dataset, [train_split, imgs_ct-train_split])
        else:
            imgs_tensor = transforms.functional.normalize(imgs_tensor, self.mean, self.std)
            self.test_set = tuple(map(lambda x, y: (x, y), imgs_tensor, labels_tensor))
            
    
    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size, shuffle=True, num_workers=4)
    
    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size, shuffle=False, num_workers=4)
    
    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size, shuffle=False)
    
    #############################
    ## Miscellaneous functions ##
    #############################
    def compute_mean_std(self, image_tensor):
        axes = (0,2,3)
        return image_tensor.mean(axis=axes), image_tensor.std(axis=axes)
        
    def get_img_path_labels(self, stage='fit'):
        data_dict = {'image_path':[], 'label':[]}
        if stage == 'fit':
            dir_ = os.path.join(self.root, 'training', 'training')
        else:
            dir_ = os.path.join(self.root, 'validation', 'validation')
            
        for cl in self.metadata['Label']:
            class_path = os.path.join(dir_, cl)
            for img in os.listdir(class_path):
                if img.endswith(".jpg") or img.endswith(".png"):
                    fname = os.path.join(class_path, img)
                    data_dict['image_path'].append(fname)
                    data_dict['label'].append(int(cl[1]))
        
        return data_dict

    
    def get_metadata(self):
        metadata = dict()
        with open(os.path.join(self.root, 'monkey_labels.txt'), mode ='r') as file:
            csvFile = csv.reader(file)            
            headers = [r.strip() for r in  next(csvFile)]
            
            for h in headers:
                metadata[h] = []
                
            for lines in csvFile:
                for i in range(len(lines)):
                    metadata[headers[i]].append(lines[i].strip())
        return metadata

In [18]:
class MonkeyType(pl.LightningModule):
    def __init__(self, learning_rate=3e-4, num_classes=10):
        super().__init__()
        self.save_hyperparameters()
        
        self.avg_pool = nn.AdaptiveAvgPool2d((5,5))
        
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(3 * 5 * 5, 1024),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(1024),
            #nn.Linear(1024, 1024),
            #nn.ReLU(inplace=True),
            #nn.BatchNorm1d(1024),
            
            nn.Linear(1024, self.hparams.num_classes)
        )
        
    def forward(self, x):
        x = self.avg_pool(x)
        x = self.fc(x)
        return x

    def training_step(self, batch, batch_idx):
        imgs, labels = batch
        pred = self(imgs)
        loss = F.cross_entropy(pred, labels)
        return loss
    
    def validation_step(self, batch, batch_idx):
        imgs, labels = batch
        output = self(imgs)
        loss = F.cross_entropy(output, labels)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

In [23]:
pl.seed_everything(10, workers=True)
m = MonkeyType()
d = PLDataModule('./../monkeys/', img_size=128, batch_size=20, train_split=0.80)

Global seed set to 10


In [24]:
trainer = pl.Trainer(accelerator='cpu', devices=1,
                     fast_dev_run=False,
                     enable_progress_bar=True,
                     enable_checkpointing=False,
                     logger=False,
                     max_epochs=10,
                     deterministic=True,
                     check_val_every_n_epoch=1)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [25]:
trainer.fit(m, d)


  | Name     | Type              | Params
-----------------------------------------------
0 | avg_pool | AdaptiveAvgPool2d | 0     
1 | fc       | Sequential        | 90.1 K
-----------------------------------------------
90.1 K    Trainable params
0         Non-trainable params
90.1 K    Total params
0.360     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 

RuntimeError: DataLoader worker (pid(s) 8146, 8154) exited unexpectedly

In [ ]:
ds = d.train_dataloader()
dsi = iter(ds)
ds1 = next(dsi)
a, b = ds1
idx = torch.randint(0, 20, (1,)).item()
a = ((a[idx].permute(1,2,0) * d.std + d.mean) * 255).clamp(0, 255).to(torch.uint8).numpy()
print(d.metadata['Common Name'][int(b[idx])])
print(d.metadata['Latin Name'][int(b[idx])])
im.fromarray(a)